In [3]:
import pandas as pd
import calendar
import glob

#### Weather Data

In [40]:
def compile_weather_data(df: pd.DataFrame) -> pd.DataFrame:
    # keeping data only of the relevant cities (Adilabad, Nizamabad, Karimnagar, Khammam and Warangal)
    warangal_varaints = ["Warangal (U)", "Warangal (R)", "Warangal", "Warangal Rural", "Warangal Urban"]
    cities = ["Adilabad", "Nizamabad", "Karimnagar", "Khammam"]
    cols = ["district", "month", "rainfall_cumm", "rainfall_avg", "temp_min", "temp_max", "humidity_min", "humidity_max", "wind_speed_min", "wind_speed_max"]
    
    warangal_df = df.loc[df["district"].isin(warangal_varaints)]
    df = df.loc[df["district"].isin(cities)]

    # getting month out of odate attribute
    df["month"] = pd.DatetimeIndex(df["odate"], dayfirst=True).month
    df["month"] = df["month"].apply(lambda x: calendar.month_abbr[x])

    # drop unnecessary rows
    df.drop(columns=["mandal","odate"], inplace=True)

    # grouping the df by district and month and getting mean values for every month
    df = df.groupby(by=["district", "month"]).agg({
        'rainfall': ['sum', 'mean'],'temp_min':'mean', 'temp_max':'mean', 'humidity_min':'mean',
          'humidity_max':'mean', 'wind_speed_min':'mean', 'wind_speed_max':'mean'
    }).reset_index()
    df.columns = df.columns.droplevel(1)
    df.columns = cols

    # doing the same operations for warangal_df
    warangal_df["month"] = pd.DatetimeIndex(warangal_df["odate"], dayfirst=True).month
    warangal_df["month"] = warangal_df["month"].apply(lambda x: calendar.month_abbr[x])
    warangal_df.drop(columns=["mandal", "odate"], inplace=True)

    warangal_df = warangal_df.groupby(by=["district", "month"]).agg({
        'rainfall': ['sum', 'mean'],'temp_min':'mean', 'temp_max':'mean', 'humidity_min':'mean',
          'humidity_max':'mean', 'wind_speed_min':'mean', 'wind_speed_max':'mean'
    }).reset_index()
    warangal_df.columns = warangal_df.columns.droplevel(1)
    warangal_df.columns = cols

    # getting mean data for warangal
    warangal_df = warangal_df.set_index(["district"]).groupby(by="month").agg({
        'rainfall_cumm': 'sum', 'rainfall_avg':'mean','temp_min':'mean', 'temp_max':'mean', 'humidity_min':'mean',
          'humidity_max':'mean', 'wind_speed_min':'mean', 'wind_speed_max':'mean'
    }).reset_index()
    
    warangal_df["district"] = "Warangal"

    # appending warangal data back to df
    df = df.append(warangal_df)

    return df

In [29]:
# def compile_weather_data(df: pd.DataFrame) -> pd.DataFrame:
#     # keeping data only of the relevant cities (Adilabad, Nizamabad, Karimnagar, Khammam and Warangal)
#     warangal_varaints = ["Warangal (U)", "Warangal (R)", "Warangal", "Warangal Rural", "Warangal Urban"]
#     cities = ["Adilabad", "Nizamabad", "Karimnagar", "Khammam"]
    
#     warangal_df = df.loc[df["district"].isin(warangal_varaints)]
#     df = df.loc[df["district"].isin(cities)]

#     # getting month out of odate attribute
#     df["month"] = pd.DatetimeIndex(df["odate"], dayfirst=True).month
#     df["month"] = df["month"].apply(lambda x: calendar.month_abbr[x])

#     # drop unnecessary rows
#     df.drop(columns=["mandal","odate"], inplace=True)

#     # grouping the df by district and month and getting mean values for every month
#     df = df.groupby(by=["district", "month"]).mean().reset_index()

#     # doing the same operations for warangal_df
#     warangal_df["month"] = pd.DatetimeIndex(warangal_df["odate"], dayfirst=True).month
#     warangal_df["month"] = warangal_df["month"].apply(lambda x: calendar.month_abbr[x])
#     warangal_df.drop(columns=["mandal", "odate"], inplace=True)
#     warangal_df = warangal_df.groupby(by=["district", "month"]).mean().reset_index()

#     # getting mean data for warangal
#     warangal_df = warangal_df.set_index(["district"]).groupby(by="month").mean().reset_index()
#     warangal_df["district"] = "Warangal"

#     # appending warangal data back to df
#     df = df.append(warangal_df)

#     return df

In [1]:
def make_csv_weather(path: str, year: int):
    files_list = glob.glob(path + "/*.csv")
    col_names = ["district", "mandal", "odate", "rainfall", "temp_min", "temp_max", "humidity_min", "humidity_max", "wind_speed_min", "wind_speed_max"]
    main_df = pd.read_csv(files_list[0], skiprows=1, names=col_names)

    if len(files_list) != 1:
        for i in range(1, len(files_list)):
            data = pd.read_csv(files_list[i], skiprows=1, names=col_names)
            main_df = pd.concat([main_df, data])
    
    main_df.sort_values(by=["district", "mandal", "odate"], inplace=True)
    # main_df = compile_weather_data(main_df)
    main_df.to_csv(f'./Weather_Data/daily_weather_data_{year}.csv', header=True, index=False)

In [4]:
make_csv_weather('./Weather_Data/daily_weather_data_2018/', 2018)
make_csv_weather('./Weather_Data/telangana-weather-data-2019--All-2023-02-14_1908/', 2019)
make_csv_weather('./Weather_Data/telangana-weather-data-2020--All-2023-02-14_1908/', 2020)
make_csv_weather('./Weather_Data/telangana-weather-data-2021-All-2023-02-14_1907/', 2021)
make_csv_weather('./Weather_Data/telangana-weather-data-2022-All-2023-02-14_1907/', 2022)

#### Vehicle Purchase Data

In [ ]:
def make_csv_vehicles(path: str, year):
    files_list = glob.glob(path + "/*.csv")

    main_df = pd.read_csv(files_list[0], encoding='utf-8')
    for i in range(1, len(files_list)):
        data = pd.read_csv(files_list[i], encoding='utf-8')
        main_df = pd.concat([main_df, data])
    
    main_df.sort_values(by="fromdate", inplace=True)
    main_df.to_csv(f'./Vehicle_Data/vehicle_purchase_data_{year}.csv', header=True, index=False)
    

In [ ]:
make_csv_vehicles("./Vehicle_Data/2019/", 2019)
make_csv_vehicles("./Vehicle_Data/2020/", 2020)
make_csv_vehicles("./Vehicle_Data/2021/", 2021)
make_csv_vehicles("./Vehicle_Data/2022/", 2022)

#### Industrial Consumption Data

In [42]:
def compile_industry_data(df: pd.DataFrame) -> pd.DataFrame:
    # dropping unnecessary columns
    df.drop(columns=["Division", "SubDivision", "Section", "Area", "CatCode", "CatDesc", "TotServices", "BilledServices"], inplace=True)

    # keeping only required cities
    df = df.loc[df["Circle"].isin(["ADILABAD", "NIZAMABAD", "KARIMNAGAR", "KHAMMAM", "WARANGAL"])]

    # grouping by district/circle and month and finding total energy consumption
    df = df.groupby(by=["Circle", "Month"]).sum().reset_index()

    return df

In [43]:
def make_csv_industry(path: str, year: int):
    files_list = glob.glob(path + "/*.csv")

    main_df = pd.read_csv(files_list[0])
    main_df["Month"] = files_list[0][67:70]
    for i in range(1, len(files_list)):
        data = pd.read_csv(files_list[i])
        data["Month"] = files_list[i][67:70]
        main_df = pd.concat([main_df, data])
    
    main_df = compile_industry_data(main_df)

    main_df['Circle'] = main_df['Circle'].apply(lambda x: x.title())
    main_df['Month'] = main_df['Month'].apply(lambda x: x.title())

    main_df.to_csv(f'./Industry_Consumption/industrial_consumption_data_{year}.csv', header=["district", "month", "units", "load"], index=False)


In [44]:
make_csv_industry('./Industry_Consumption/2019/', 2019)
make_csv_industry('./Industry_Consumption/2020/', 2020)
make_csv_industry('./Industry_Consumption/2021/', 2021)
make_csv_industry('./Industry_Consumption/2022/', 2022)

#### AQI Data

In [9]:
def compile_aqi_data(sheet_name: str) -> None:
    df = pd.read_excel('./AQI_Data/aqi_data.xlsx', sheet_name=sheet_name, index_col=None)

    # taking average of 2 monitoring stations in warangal
    warangal_df = df.loc[df["Location"].isin(["Kuda, warangal", "Mee-Seva, Warangal"])].drop(columns="Location").mean()
    warangal_df['Location'] = "Warangal"

    df = df.loc[df["Location"].isin(["Nizamabad", "Adilabad", "Karimnagar", "Khammam"])]
    df = df.append(warangal_df, ignore_index=True)

    # taking transpose of df
    df = df.T

    # making first row as the header and dropping it
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)

    # flattening 2D df to 1D with index as Location and Month
    df = df.T.stack().reset_index()

    df.to_csv(f'./AQI_Data/monthly_aqi_data_{sheet_name}.csv', header=["district", "month", "aqi"], index=False)

In [10]:
compile_aqi_data(sheet_name="2017")
compile_aqi_data(sheet_name="2018")
compile_aqi_data(sheet_name="2019")
compile_aqi_data(sheet_name="2020")
compile_aqi_data(sheet_name="2021")
compile_aqi_data(sheet_name="2022")

#### AAQ Data

In [15]:
def compile_aaq_data(df: pd.DataFrame, pollutant:str) -> pd.DataFrame:
    adilabad_variants = ['adilabad', 'mandamarri']
    nizamabad_variants = ['nizamabad', 'subhasnagar']
    khammam_variants = ['khammam', 'Jalasouda']
    warangal_variants = ['warangal', 'kuda', 'mee-seva']
    karimnagar_variants = ['karimnagar', 'godavarikhani', 'DIC building']

    # getting pattern for warangal and taking avg of the variants
    warangal_pattern = '|'.join(warangal_variants)
    warangal_df = df.loc[df.location.str.contains(warangal_pattern, case=False)].drop(columns='location').mean()
    warangal_df['location'] = 'Warangal'

    # doing the same for the karimnagar variants
    karimnagar_pattern = '|'.join(karimnagar_variants)
    karimnagar_df = df.loc[df.location.str.contains(karimnagar_pattern, case=False)].drop(columns='location').mean()
    karimnagar_df['location'] = 'Karimnagar'

    # keeping other required cities apart from karimnagar and warangal
    location_pattern = '|'.join(adilabad_variants + nizamabad_variants + khammam_variants)
    df = df.loc[df.location.str.contains(location_pattern, case=False)]
    df = df.append([karimnagar_df, warangal_df], ignore_index=True)   

    # stacking the dataframe
    df = df.set_index("location").stack().reset_index()
    df.columns = ["district", "month", pollutant]

    # renaming rows to have standard district names
    df.loc[df['district'].str.contains('|'.join(adilabad_variants), case=False), 'district'] = "Adilabad"
    df.loc[df['district'].str.contains('|'.join(nizamabad_variants), case=False), 'district'] = "Nizamabad"
    df.loc[df['district'].str.contains('|'.join(khammam_variants), case=False), 'district'] = "Khammam"

    return df

In [34]:
def make_aaq_csv(file_path: str):
    # files_list = glob.glob('./AAQ_Data/' + '/*.xls')
    year = file_path[-8:-4]
    col_names = ["location", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    sheet_names = ["SO2", "NOx", "PM10", "PM2.5", "NH3"]
    

    main_df = pd.read_excel(file_path, sheet_name=sheet_names[0], header=5, usecols='C:O', names=col_names)
    main_df = main_df[main_df['location'].notna()]
    main_df[main_df.columns.drop('location')] = main_df[main_df.columns.drop('location')].apply(pd.to_numeric, errors='coerce')
    main_df = compile_aaq_data(main_df, sheet_names[0])

    for sheet in sheet_names[1:]:
        df = pd.read_excel(file_path, sheet_name=sheet, header=5, usecols='C:O', names=col_names)
        df = df[df['location'].notna()]
        df[df.columns.drop('location')] = df[df.columns.drop('location')].apply(pd.to_numeric, errors='coerce')
        df = compile_aaq_data(df, sheet)
        main_df = pd.merge(main_df, df, on=['district', 'month'], how='outer')

    main_df.to_csv(f'./AAQ_Data/monthly_aaq_data_{year}.csv', header=True, index=False)

In [36]:
files_list = glob.glob('./AAQ_Data/' + '/*.xls')
for file in files_list:
    make_aaq_csv(file)


#### Combining CSVs

In [6]:
def combine_csv(path: str, data: str) -> None:
    # files_list = glob.glob(path + "/*.csv")
    files_list = glob.glob(path + "/daily*.csv")

    main_df = pd.read_csv(files_list[0])

    # comment below line for daily results
    # main_df["year"] = files_list[0][-8:-4]
    for i in range(1, len(files_list)):
        df = pd.read_csv(files_list[i])
        # df["year"] = files_list[i][-8:-4]

        main_df = pd.concat([main_df, df])
    
    # main_df.to_csv(f"./monthly_{data}_data.csv", header=True, index=False)
    main_df.to_csv(f"./daily_{data}_data.csv", header=True, index=False)

In [12]:
combine_csv("./AQI_Data/", "aqi")
combine_csv("./Industry_Consumption/", "industry_consumption")


In [ ]:
combine_csv("./Weather_Data/", "weather")

In [39]:
combine_csv("./AAQ_Data/", "aaq")

In [ ]:
combine_csv('./Weather_Data/', 'weather')